## Importando bibliotecas

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, LassoCV, ElasticNetCV
from sklearn.feature_selection import RFE, RFECV
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from os import listdir

## funções

In [2]:
## função de scoring personalizado para algoritmos de seleção de feature do sklearn
def r2_score_adj(estimator, X, y):
    
    number_of_samples = X.shape[0]
    number_of_preditors = X.shape[1]
    
    ## caso inválido (denominador = 0)
    if ((number_of_samples - number_of_preditors - 1)==0):
            return 0
    
    ## valores preditos
    y_pred = estimator.predict(X)
    
    ## definição do R²
    rsquared = r2_score(y,y_pred)
    
    return 1 - (number_of_samples - 1) / (number_of_samples - number_of_preditors - 1) * (1 - rsquared)


## função para a combinação RFE (otimizada por RMSE) e regressão linear 
def m0_model(X_train, y_train, X_test, y_test):
    lr = LinearRegression().fit(X_train,y_test)
    model = RFECV(estimator=lr, scoring='neg_root_mean_squared_error')
    model.fit(X_train,y_train)
    return model

## função para a combinação RFE (otimizada por R² ajustado) e regressão linear 
def m1_model(X_train, y_train, X_test, y_test):
    lr = LinearRegression().fit(X_train,y_test)
    model = RFECV(estimator=lr, scoring=r2_score_adj)
    model.fit(X_train,y_train)
    return model

## função para a combinação Regressão linear com regularização Lasso (alfa definido por cross-validation) 
def m2_model(X_train, y_train, X_test, y_test):
    model = LassoCV(alphas = [0.1,0.11,0.12,0.13,0.14,0.15,0.20,0.25])
    model.fit(X_train, y_train)
    return model

## função para a combinação Regressão linear com regularização Ridge (alfa definido por cross-validation) 
def m3_model(X_train, y_train, X_test, y_test):
    alphas = 10**np.linspace(10,-2,100)*0.5
    model = RidgeCV(alphas = alphas)
    model.fit(X_train, y_train.to_numpy().ravel())
    return model

## função para a combinação RFE (otimizada por RMSE) e floresta aleatória ) 
def m4_model(X_train, y_train, X_test, y_test):
    
    rf = RandomForestRegressor(random_state=0)

    #rf.fit(X_train,y_train.to_numpy().ravel())

    rfe = RFECV(rf,cv=5,scoring="neg_root_mean_squared_error")

    rfe.fit(X_train,y_train.to_numpy().ravel())

    return rfe

def get_results(model_function, scaling_y, X, y):
    
    metrics_dict = {}
    
    
    
    cv_r2 = {'train':[],'test':[]}
    cv_r2a = {'train':[],'test':[]}
    cv_rmse = {'train':[],'test':[]}
    cv_mae = {'train':[],'test':[]}
    cv_in_std = {'train':[],'test':[]}

    kf = KFold(n_splits=2)
    KFold(n_splits=5, random_state=42, shuffle=True)
    
    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]
        
        ## obtendo as predições
        model = model_function(X_train, y_train, X_test, y_test)
        
        ## caso seja um modelo com redução de variáveis
        if type(model) == tuple:
            X_columns = model[1]
            X_train = X_train[X_columns]
            X_test = X_test[X_columns]
            model = model[0]
            
        train_preds = model.predict(X_train)
        test_preds = model.predict(X_test)

        ## obtendo o y desnormalizado
        y_train_denorm = scaling_y.inverse_transform(y_train)
        y_train_denorm_np = scaling_y.inverse_transform(y_train.to_numpy().reshape(-1, 1)) ## para funções que precisem  do formato numpy
        train_preds_denorm = scaling_y.inverse_transform(train_preds.reshape(-1, 1))
        ##
        y_test_denorm = scaling_y.inverse_transform(y_test)
        y_test_denorm_np = scaling_y.inverse_transform(y_test.to_numpy().reshape(-1, 1)) ## para funções que precisem  do formato numpy
        test_preds_denorm = scaling_y.inverse_transform(test_preds.reshape(-1, 1))
        
        ## calculando as métricas de treinamento
        r2 = r2_score(y_train_denorm, train_preds_denorm)
        n = len(y_train)
        p = X_train.shape[1]
        adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
        mse = mean_squared_error(y_train_denorm, train_preds_denorm)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_train_denorm, train_preds_denorm)
        y_in_std = np.std(y_train_denorm_np)
        
        ## salvando nos cross_validation_dicts:
        cv_r2['train'].append(r2)
        cv_r2a['train'].append(adj_r2)
        cv_rmse['train'].append(rmse)
        cv_mae['train'].append(mae)
        cv_in_std['train'].append(y_in_std)

        ## calculando as métricas de teste
        r2 = r2_score(y_test_denorm, test_preds_denorm)
        n = len(y_test)
        p = X_test.shape[1]
        adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
        mse = mean_squared_error(y_test_denorm, test_preds_denorm)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_test_denorm, test_preds_denorm)
        y_in_std = np.std(y_test_denorm_np)
             
        ## salvando nos cross_validation_dicts:
        cv_r2['test'].append(r2)
        cv_r2a['test'].append(adj_r2)
        cv_rmse['test'].append(rmse)
        cv_mae['test'].append(mae)
        cv_in_std['test'].append(y_in_std)
    
    metrics_dict['r2_treinamento'] = np.mean(cv_r2['train'])
    metrics_dict['r2_ajustado_treinamento'] = np.mean(cv_r2a['train'])
    metrics_dict['rmse_treinamento'] = np.mean(cv_rmse['train'])
    metrics_dict['mae_treinamento'] = np.mean(cv_mae['train'])
    metrics_dict['y_in_std_treinamento'] = np.mean(cv_in_std['train'])
    
    metrics_dict['r2_teste'] = np.mean(cv_r2['test'])
    metrics_dict['r2_ajustado_teste'] = np.mean(cv_r2a['test'])
    metrics_dict['rmse_teste'] = np.mean(cv_rmse['test'])
    metrics_dict['mae_teste'] = np.mean(cv_mae['test'])
    metrics_dict['y_in_std_teste'] = np.mean(cv_in_std['test'])
    

    metrics_dict['summary_text'] = summary_text(metrics_dict)
    return metrics_dict

def summary_text(metrics_dict):
    return """
R² de treinamento: {:.5f}
R² ajustado de treinamento: {:.5f}
RMSE de treinamento: {:.5f}
MAE de treinamento: {:.5f}
STD input de treinamento: {:.5f}

R² de teste: {:.5f}
R² ajustado de teste: {:.5f}
RMSE de teste: {:.5f}
MAE de teste: {:.5f}
STD input de teste: {:.5f}
    """.format(metrics_dict['r2_treinamento'], metrics_dict['r2_ajustado_treinamento'], metrics_dict['rmse_treinamento'], metrics_dict['mae_treinamento'], metrics_dict['y_in_std_treinamento'], metrics_dict['r2_teste'], metrics_dict['r2_ajustado_teste'], metrics_dict['rmse_teste'], metrics_dict['mae_teste'], metrics_dict['y_in_std_teste'])


def super_func(df_path, y_column_name):
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")
    print('Avaliando o arquivo: ',df_path)
    print('Modelos a serem testados: ')
    model_names = ['Seleção de feature por RFECV (scoring RMSE) + Regressão linear','Seleção de feature por RFECV (scoring R2a) + Regressão linear','Lasso','Ridge','Seleção de feature por RFE (RMSE/iterativa) + Floresta aleatória']
    print(model_names)
    
    ## lendo o arquivo
    df_original = pd.read_csv('./'+df_path)
    
    ## corrigindo , para .
    df_original = df_original.applymap(lambda x: x if type(x) == type(0.0) else str(x).replace(",","."))
    
    df = df_original.copy()
    
    ## corigindo espaços sobrando nos nomes das colunas
    df.columns = [col.strip() for col in df.columns]
    
    ## garantindo que os dados estão no formato numérico 
    for col in df.columns[:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    
    ## definindo que as colunas à esquerda da coluna resposta são atributos
    out_index = list(df.columns).index(y_column_name)
    X_columns = df.columns[:out_index]
    
    ############## fazendo o scaling dos dados #####################
    scaling_x = StandardScaler()
    scaling_y = StandardScaler()
    X = pd.DataFrame(scaling_x.fit_transform(df[X_columns]))
    X.columns = X_columns
    y = pd.DataFrame(scaling_y.fit_transform(df[[y_column_name]]))
    y.columns = [y_column_name]

    ############ fazendo o split de dados #######################
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)
    
    row = {'Dados':df_path}
    
    print('##########################')
    print(model_names[0])
    row[model_names[0]] = get_results(m0_model, scaling_y, X, y)['summary_text']
    print(row[model_names[0]])
    
    print('##########################')
    print(model_names[1])
    row[model_names[1]] = get_results(m1_model, scaling_y, X, y)['summary_text']
    print(row[model_names[1]])
    
    print('##########################')
    print(model_names[2])
    row[model_names[2]] = get_results(m2_model, scaling_y, X, y)['summary_text']
    print(row[model_names[2]])
    
    print('##########################')
    print(model_names[3])
    row[model_names[3]] = get_results(m3_model, scaling_y, X, y)['summary_text']
    print(row[model_names[3]])
    
    print('##########################')
    print(model_names[4])
    row[model_names[4]] = get_results(m4_model, scaling_y, X, y)['summary_text']
    print(row[model_names[4]])
    
    return row



## chamada

In [3]:
df_path_list = ['NOX2 - 4.csv']
final_models_df = pd.DataFrame(columns = ['Dados','Seleção de feature por RFECV (scoring RMSE) + Regressão linear','Seleção de feature por RFECV (scoring R2a) + Regressão linear','Seleção de feature por RFE (RMSE/iterativa) + Regressão linear','Seleção de feature por RFE (R2a/iterativa) + Regressão linear','Lasso','Ridge','RIDGE (Seleção manual por R2 ajustado)','Seleção de feature por RFE (RMSE/iterativa) + Floresta aleatória'])

for df_path in df_path_list:
    final_models_df = final_models_df.append(super_func(df_path,'NOx (ppm)'), ignore_index=True)

final_models_df.to_csv('./LAST RUN OUTPUT.csv')
display(final_models_df)

´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
Avaliando o arquivo:  NOX2 - 4.csv
Modelos a serem testados: 
['Seleção de feature por RFECV (scoring RMSE) + Regressão linear', 'Seleção de feature por RFECV (scoring R2a) + Regressão linear', 'Lasso', 'Ridge', 'Seleção de feature por RFE (RMSE/iterativa) + Floresta aleatória']
##########################
Seleção de feature por RFECV (scoring RMSE) + Regressão linear

R² de treinamento: 0.23616
R² ajustado de treinamento: 0.19536
RMSE de treinamento: 124.82259
MAE de treinamento: 91.29252
STD input de treinamento: 142.50592

R² de teste: -11.13935
R² ajustado de teste: -11.78782
RMSE de teste: 441.64509
MAE de teste: 111.00757
STD input de teste: 142.50592
    
##########################
Seleção de feature por RFECV (scoring R2a) + Regressão linear

R² de treinamento: 0.23616
R² ajustado de treinamento: 0.19536
RMSE de treinamento: 124.82259
MAE de treinamento: 91.29252
STD input de treinamento: 142

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)



R² de treinamento: 0.93371
R² ajustado de treinamento: 0.93017
RMSE de treinamento: 36.67950
MAE de treinamento: 24.54153
STD input de treinamento: 142.50592

R² de teste: 0.21740
R² ajustado de teste: 0.17560
RMSE de teste: 126.21240
MAE de teste: 90.57147
STD input de teste: 142.50592
    


,Dados,Seleção de feature por RFECV (scoring RMSE) + Regressão linear,Seleção de feature por RFECV (scoring R2a) + Regressão linear,Seleção de feature por RFE (RMSE/iterativa) + Regressão linear,Seleção de feature por RFE (R2a/iterativa) + Regressão linear,Lasso,Ridge,RIDGE (Seleção manual por R2 ajustado),Seleção de feature por RFE (RMSE/iterativa) + Floresta aleatória
0,NOX2 - 4.csv,\nR² de treinamento: 0.23616\nR² ajustado de t...,\nR² de treinamento: 0.23616\nR² ajustado de t...,NaN,NaN,\nR² de treinamento: 0.33864\nR² ajustado de t...,\nR² de treinamento: 0.53527\nR² ajustado de t...,NaN,\nR² de treinamento: 0.93371\nR² ajustado de t...
